In [1]:
from os import listdir, walk
from os.path import isfile, join
import numpy as np
from PIL import Image
import time
import cv2
from sys import getsizeof
from IPython.display import display
import random
import math

from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import Input
from tensorflow.keras.utils import Sequence

%run ./variables.ipynb

# Handle data

In [9]:
# using Keras sequence
class DiatomSequence(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return np.array([cv2.imread(file_name) for file_name in batch_x]), np.array(batch_y)

In [10]:
start_time = time.time()
x_set = []
y_set = []
taxons_dirs = next(walk(DATASET_PATH))[1]
n_taxons = len(taxons_dirs)
disp_progress = display("0/"+str(n_taxons),display_id=True)
for i, taxon in enumerate(taxons_dirs):
    path = join(DATASET_PATH, taxon)
    files = [f for f in listdir(path) if isfile(join(path, f))]
    for file in files:
        x_set.append(join(path, file))
        y_set.append(taxon)
    disp_progress.update(str(i)+"/"+str(n_taxons))
# Shuffling
xy = list(zip(x_set, y_set))
random.shuffle(xy)
x_set, y_set = zip(*xy)
print("Job finished in ", (time.time()-start_time), "s!")

'184/185'

Job finished in  0.45839881896972656 s!


In [11]:
dataset = DiatomSequence(x_set, y_set, 50)

# Model

In [12]:
model = Xception(include_top=False, weights='imagenet', input_shape=(256, 256, 3), pooling=None, classes=185)

In [13]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [14]:
model.fit(dataset)

  ...
    to  
  ['...']
Train for 740 steps
  1/740 [..............................] - ETA: 1:00:46

UnimplementedError:  Cast string to float is not supported
	 [[node loss/block14_sepconv2_act_loss/Cast (defined at <ipython-input-14-d28793719248>:1) ]] [Op:__inference_distributed_function_20502]

Function call stack:
distributed_function


NameError: name 'SELECTED_TAXONS' is not defined